### Stock Database Analysis

In [1]:
from stocksense.database_handler import DatabaseHandler

db = DatabaseHandler()

2024-11-28 15:50:25.454 | SUCCESS  | stocksense.database_handler.schema:create_tables:121 - Tables created successfully


In [2]:
import polars as pl

from stocksense.pipeline.scraper import Scraper

base_df = pl.read_csv(
    "../stocksense/data/interim/SP500.csv", separator=";", columns=["tic", "name", "sector"]
)
curr_df = Scraper.scrape_sp500_constituents()
base_df

tic,sector,name
str,str,str
"""AABA""","""Financials""",null
"""AAP""","""Consumer Discretionary""",null
"""ABC""","""Health Care""",null
"""ABI""","""Industrials""",null
"""ABKFQ""","""Financials""",null
…,…,…
"""XYL""","""Industrials""","""Xylem Inc."""
"""YUM""","""Consumer Discretionary""","""Yum! Brands"""
"""ZBRA""","""Information Technology""","""Zebra Technologies"""


In [3]:
additions, removals = Scraper.scrape_sp500_changes()
additions

added,tic,name_added
date,str,str
2024-11-26,"""TPL""","""Texas Pacific Land Corporation"""
2024-09-30,"""AMTM""","""Amentum"""
2024-09-23,"""PLTR""","""Palantir Technologies"""
2024-09-23,"""DELL""","""Dell Technologies"""
2024-09-23,"""ERIE""","""Erie Indemnity"""
…,…,…
1999-06-09,"""WLP""","""Wellpoint"""
1998-12-11,"""FSR""","""Firstar"""
1998-12-11,"""CCL""","""Carnival Corp."""


In [4]:
df = base_df.join(additions, on="tic", how="left")
df = df.join(removals, on="tic", how="left")
df = df.join(curr_df.select(["tic", "date_added"]), on="tic", how="left")
df = df.with_columns(pl.col("removed").alias("date_removed"))
df = df.with_columns(
    pl.when(pl.col("date_added").is_null())
    .then(pl.col("added"))
    .otherwise(pl.col("date_added"))
    .alias("date_added")
).with_columns(
    pl.when(pl.col("name").is_null())
    .then(pl.col("name_removed"))
    .otherwise(pl.col("name"))
    .alias("name")
)
df = df.filter(~(pl.col("date_added").is_null() & pl.col("date_removed").is_null()))

df = df[["tic", "name", "sector", "date_added", "date_removed"]]

In [4]:
stocks = db.fetch_stock("AMZN")
stocks

tic,name,sector,date_added,date_removed
str,str,str,date,date
"""AMZN""","""Amazon""","""Consumer Discretionary""",2005-11-18,null


In [5]:
mdf = db.fetch_market_data("AMZN").sort(by=["date"])
mdf

tic,date,close,adj_close,volume
str,date,f64,f64,i64
"""AMZN""",2005-01-03,2.226,2.226,208930000
"""AMZN""",2005-01-04,2.107,2.107,388370000
"""AMZN""",2005-01-05,2.0885,2.0885,167084000
"""AMZN""",2005-01-06,2.0525,2.0525,174018000
"""AMZN""",2005-01-07,2.116,2.116,196732000
…,…,…,…,…
"""AMZN""",2024-10-25,187.830002,187.830002,29362100
"""AMZN""",2024-10-28,188.389999,188.389999,27930800
"""AMZN""",2024-10-29,190.830002,190.830002,35690200


In [7]:
db.fetch_insider_data("PLTR").to_pandas()

,tic,filling_date,trade_date,owner_name,title,transaction_type,last_price,qty,shares_held,owned,value
0,PLTR,2020-10-02,2020-09-30,Cohen Stephen Andrew,"Pres, Secretary",S - Sale+OE,$9.71,"-2,000,000","2,402,647",-45%,"-$19,422,100"
1,PLTR,2020-10-02,2020-09-30,Glazer David A.,"CFO, Treasurer",S - Sale,$10.03,"-1,615,621","3,940,152",-29%,"-$16,206,016"
2,PLTR,2020-10-02,2020-09-30,Karp Alexander C.,CEO,S - Sale+OE,$9.70,"-11,500,000","9,026,496",-56%,"-$111,606,220"
3,PLTR,2020-10-02,2020-09-30,Long Matthew A.,GC,S - Sale+OE,$10.06,"-579,404","874,006",-40%,"-$5,830,660"
4,PLTR,2020-10-02,2020-09-30,Moore Alexander D.,Dir,S - Sale,$9.54,"-285,000","2,429,249",-11%,"-$2,719,315"
...,...,...,...,...,...,...,...,...,...,...,...
277,PLTR,2024-10-07,2024-10-03,Planishek Heather A.,See Remarks,S - Sale+OE,$39.32,"-34,017","570,136",-6%,"-$1,337,500"
278,PLTR,2024-10-08,2024-10-04,Taylor Ryan D.,See Remarks,S - Sale+OE,$40.00,"-195,500","331,369",-37%,"-$7,820,704"
279,PLTR,2024-10-15,2024-10-11,Stat Lauren Elaina Friedman,Dir,S - Sale,$43.90,"-7,321","201,243",-4%,"-$321,392"
280,PLTR,2024-10-29,2024-10-25,Karp Alexander C.,See Remarks,S - Sale+OE,$45.01,"-5,656,293","6,432,258",-47%,"-$254,610,669"


In [6]:
db.fetch_financial_data("AMZN")

tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct
str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""AMZN""",2005-03-31,2005-04-26,1902.0,1416.0,342.0,78.0,144.0,411.0,1793.0,2472.0,1151.0,null,403.0,245.0,1073.0,1561.0,2634.0,-2284.0,-162.0,-294.0,-197.0,-256.0,0.0,26.0,1399.0,-0.209
"""AMZN""",2005-06-30,2005-07-26,1753.0,1275.0,350.0,52.0,128.0,412.0,1926.0,2601.0,1325.0,null,383.0,267.0,1144.0,1521.0,2665.0,-2229.0,-64.0,244.0,-144.0,9.0,0.0,47.0,1457.0,0.138
"""AMZN""",2005-09-30,2005-11-29,1858.0,1365.0,380.0,30.0,113.0,414.0,2121.0,2832.0,1419.0,null,456.0,322.0,1313.0,1513.0,2826.0,-2213.0,6.0,153.0,-206.0,30.0,0.0,76.0,1519.0,null
"""AMZN""",2005-12-31,2006-02-02,2977.0,2274.0,502.0,199.0,201.0,416.0,2929.0,3696.0,2000.0,null,566.0,348.0,1929.0,1521.0,3450.0,-2021.0,246.0,630.0,-231.0,24.0,0.0,55.0,1767.0,1.209
"""AMZN""",2006-03-31,2006-04-25,2279.0,1692.0,441.0,51.0,146.0,417.0,2186.0,2990.0,1334.0,null,538.0,361.0,1407.0,1259.0,2666.0,-1967.0,324.0,-303.0,84.0,-296.0,0.0,46.0,1583.0,0.024
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""AMZN""",2023-06-30,2023-08-03,134383.0,112609.0,13947.0,6750.0,19992.0,10313.0,140482.0,477607.0,63970.0,39925.0,36587.0,264116.0,148238.0,63092.0,309005.0,93115.0,168602.0,16476.0,-9673.0,-6539.0,null,11455.0,231694.0,0.857
"""AMZN""",2023-09-30,2023-10-26,143083.0,118539.0,13112.0,9879.0,24044.0,10330.0,142995.0,486883.0,64169.0,38100.0,35406.0,267226.0,145214.0,61098.0,303910.0,102994.0,182973.0,21217.0,-11753.0,-8948.0,null,12479.0,244071.0,0.608
"""AMZN""",2023-12-31,2024-02-01,169961.0,140686.0,15912.0,10624.0,27949.0,10383.0,172351.0,527854.0,86780.0,52253.0,33318.0,276690.0,164917.0,58314.0,325979.0,113618.0,201875.0,42465.0,-12601.0,-6746.0,null,14588.0,260189.0,0.246


In [9]:
st = db.fetch_stock().to_pandas()
st

,tic,name,sector,date_added,date_removed
0,AAP,Advance Auto Parts,Consumer Discretionary,2015-07-08,2023-08-25
1,ABMD,Abiomed,Health Care,2018-05-31,2022-12-22
2,ABS,Albertsons,Consumer Staples,NaT,2006-06-02
3,ACAS,American Capital,Financials,NaT,2009-03-03
4,ADCT,ADC Telecommunications,Communication Services,NaT,2007-07-02
...,...,...,...,...,...
783,XYL,Xylem Inc.,Industrials,2011-11-01,NaT
784,YUM,Yum! Brands,Consumer Discretionary,1997-10-06,NaT
785,ZBRA,Zebra Technologies,Information Technology,2019-12-23,NaT
786,ZBH,Zimmer Biomet,Health Care,2001-08-07,NaT


In [10]:
inf = db.fetch_info("AAPL")
inf

,tic,risk,beta,volume,trailing_pe,forward_pe,trailing_peg,peg,trailing_eps,forward_eps,...,currency,shares_outstanding,enterprise_value,short_ratio,curr_price,fiftytwo_wc,rec_key,target_low,target_high,target_mean
0,AAPL,1,1.24,47923696.0,37.00658,27.079428,2.3294,None,6.08,8.31,...,USD,1.511580e+10,3.454948e+12,3.24,225.0,0.175242,None,184.0,300.0,244.47739


In [11]:
inf.loc[0, ["risk"]].values[0]

np.int64(1)